# Feature Selection using Correlation

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("D:/Python DataScience/Feature Selection/santander-train.csv",nrows=20000)

In [ ]:
X = df.drop(['TARGET'],axis=1)
y = df['TARGET']

In [ ]:
X.shape,y.shape

In [ ]:
X_train ,X_test ,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0,stratify=y)

In [ ]:
from sklearn.feature_selection import VarianceThreshold

In [ ]:
constant_filter = VarianceThreshold(threshold=0)
constant_filter.fit(X_train)
X_train_filter = constant_filter.transform(X_train)
X_test_filter = constant_filter.transform(X_test)

In [ ]:
X_train_filter.shape,X_test_filter.shape

In [ ]:
quasi_constant_filter = VarianceThreshold(threshold=0.01)
X_train_quasifilter = quasi_constant_filter.fit_transform(X_train_filter)
X_test_quasifilter = quasi_constant_filter.transform(X_test_filter)

In [ ]:
X_train_quasifilter.shape,X_test_quasifilter.shape

In [ ]:
X_train_T = X_train_quasifilter.T
X_test_T = X_test_quasifilter.T
X_train_T = pd.DataFrame(X_train_T)
X_test_T = pd.DataFrame(X_test_T)

In [ ]:
X_train_T.shape , X_test_T.shape

In [ ]:
duplicated_features = X_train_T.duplicated()
non_duplicated_features = [not i for i in duplicated_features]

In [ ]:
X_train_unique = X_train_T[non_duplicated_features].T
X_test_unique = X_test_T[non_duplicated_features].T

In [ ]:
X_train_unique.shape,X_test_unique.shape

## Use of Pearson Correlation Coefficient

We will take correlation factor as 85%.
If two features are more then 85% correlated we drop one of the feature.

In [ ]:
corr_matrix = X_train_unique.corr()
plt.figure(figsize=(12,8))
sns.heatmap(corr_matirx)

### Getting out the list of all correlated columns

In [ ]:
def get_corr(data,threshold):
    corr_col = set()
    corr_mat = data.corr()
    for i in range(len(corr_mat.columns)):
        for j in range(i):
            if abs(corr_mat.iloc[i,j])>threshold:
                col_name = corr_mat.columns[i]
                corr_col.add(col_name)
                
    return list(corr_col)       

In [ ]:
corr_list = get_corr(X_train_unique,0.85)

In [ ]:
len(corr_list)

In [ ]:
X_train_uncorr = X_train_unique.drop(corr_list,axis=1)
X_test_uncorr = X_test_unique.drop(corr_list,axis=1)

In [ ]:
X_train_uncorr.shape,X_test_uncorr.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
def run_randomForest(X_train,X_test,y_train,y_test):
    clf = RandomForestClassifier(n_estimators=100,random_state=0,n_jobs=-1)
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    print("Accuracy is : ",accuracy_score(y_test,y_pred))

In [124]:
%%time
run_randomForest(X_train_uncorr,X_test_uncorr,y_train,y_test)

Accuracy is :  0.95875
Wall time: 1.72 s


In [125]:
%%time
run_randomForest(X_train,X_test,y_train,y_test)

Accuracy is :  0.9585
Wall time: 3.3 s


## Feature Importance and Feature Grouping

In [66]:
corr_matrix

,0,1,2,3,4,5,6,7,8,9,...,235,236,237,238,239,240,241,242,243,244
0,1.000000,-0.025277,-0.001942,0.003594,0.004054,-0.001697,-0.015882,-0.019807,0.000956,-0.000588,...,-0.001337,0.002051,-0.008500,0.006554,0.005907,0.008825,-0.009174,0.012031,0.012128,0.006612
1,-0.025277,1.000000,-0.007647,0.001819,0.008981,0.009232,0.001638,0.001746,0.000614,0.000695,...,0.000544,0.000586,0.000337,0.000550,0.000563,0.000922,0.000598,0.000875,0.000942,0.000415
2,-0.001942,-0.007647,1.000000,0.030919,0.106245,0.109140,0.048524,0.055708,0.004040,0.005796,...,0.025522,0.020168,0.011550,0.019325,0.019527,0.041321,0.016172,0.043577,0.044281,-0.000810
3,0.003594,0.001819,0.030919,1.000000,0.029418,0.024905,0.014513,0.013857,-0.000613,-0.000691,...,0.014032,-0.000583,-0.000337,-0.000548,-0.000561,0.000541,-0.000577,0.000231,0.000235,0.000966
4,0.004054,0.008981,0.106245,0.029418,1.000000,0.888789,0.381632,0.341266,0.012927,0.019674,...,0.002328,0.016743,-0.001662,0.020509,0.021276,-0.001905,-0.000635,-0.002552,-0.002736,0.003656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,0.008825,0.000922,0.041321,0.000541,-0.001905,0.000871,-0.000818,-0.000866,-0.000309,-0.000349,...,0.012705,0.021540,-0.000170,0.032162,0.030087,1.000000,0.329805,0.935317,0.919036,0.011106
241,-0.009174,0.000598,0.016172,-0.000577,-0.000635,0.007096,-0.000515,-0.000545,-0.000195,-0.000220,...,-0.000173,-0.000185,-0.000107,-0.000174,-0.000178,0.329805,1.000000,0.127224,0.140902,0.011807
242,0.012031,0.000875,0.043577,0.000231,-0.002552,-0.001672,-0.000779,-0.000825,-0.000295,-0.000332,...,0.027515,0.012393,-0.000162,0.018565,0.017358,0.935317,0.127224,1.000000,0.993536,0.008604
243,0.012128,0.000942,0.044281,0.000235,-0.002736,-0.001844,-0.000839,-0.000888,-0.000317,-0.000358,...,0.023072,0.014523,-0.000174,0.021742,0.020331,0.919036,0.140902,0.993536,1.000000,0.009136


In [79]:
corr_data = corr_matrix.abs().stack()

In [68]:
len(corr_data) # all the feature are stacked

51529

In [80]:
corr_data

0    0      1.000000
     1      0.025277
     2      0.001942
     3      0.003594
     4      0.004054
              ...   
244  240    0.011106
     241    0.011807
     242    0.008604
     243    0.009136
     244    1.000000
Length: 51529, dtype: float64

In [81]:
corr_data = corr_data.sort_values(ascending=False)

In [82]:
corr_data

29   58     1.000000e+00
58   29     1.000000e+00
134  158    1.000000e+00
158  134    1.000000e+00
182  182    1.000000e+00
                ...     
229  111    1.934954e-06
231  150    6.044672e-07
150  231    6.044672e-07
231  123    3.966696e-07
123  231    3.966696e-07
Length: 51529, dtype: float64

In [83]:
corr_data = corr_data[corr_data>0.85]
corr_data = corr_data[corr_data<1]

In [84]:
corr_data

143  135    1.000000
135  143    1.000000
136  128    1.000000
128  136    1.000000
31   62     1.000000
              ...   
67   66     0.851384
61   28     0.851022
28   61     0.851022
72   35     0.850893
35   72     0.850893
Length: 534, dtype: float64

In [85]:
corr_data = pd.DataFrame(corr_data).reset_index()
corr_data.columns = ['Features1','Features2','corr']

## Making group of correlated features

In [87]:
group_feature_list = []
correlated_groups_list = []

for feature in corr_data.Features1.unique():
    if feature not in group_feature_list:
        corr_block = corr_data[corr_data.Features1 == feature]
        group_feature_list = group_feature_list + list(corr_block.Features2.unique())+[feature]
        correlated_groups_list.append(corr_block)
        

In [90]:
len(correlated_groups_list)

56

In [91]:
for feature in correlated_groups_list:
    print(feature)

   Features1  Features2  corr
0        143        135   1.0
     Features1  Features2      corr
2          136        128  1.000000
197        136        169  0.959468
   Features1  Features2  corr
4         31         62   1.0
   Features1  Features2  corr
6         20         47   1.0
     Features1  Features2      corr
8           52         23  1.000000
297         52         24  0.927683
299         52         53  0.927683
448         52         21  0.877297
505         52        183  0.860163
     Features1  Features2      corr
12          33         69  1.000000
224         33         32  0.947113
228         33         68  0.946571
322         33         26  0.917665
337         33         55  0.914178
422         33        184  0.884383
    Features1  Features2  corr
14        157        133   1.0
    Features1  Features2      corr
16        237        149  1.000000
26        237        148  0.999929
    Features1  Features2  corr
18        154        132   1.0
     Features1 

## Feature Importance using Tree Classifier
### Extracting the most important feature in each group

In [103]:
# Intitially we create empty list for storing important features
important_features = []

# In each group we are getting the most important feature

for group in correlated_groups_list:
    # In each group we are gathering unique features
    features = list(group.Features1.unique())+list(group.Features2.unique())
    
    # USE of random forest to estimate the importance
    rf = RandomForestClassifier(n_estimators=100,random_state=0)
    rf.fit(X_train_unique[features],y_train)
    
    #creating a dataframe for important features along with their importance
    important = pd.concat([pd.Series(features),pd.Series(rf.feature_importances_)],axis=1)
    important.columns = ["Features",'importance']
    
    # sort the columns by highest importance 
    important.sort_values(by='importance',ascending = False,inplace=True)
    
    # from each group taking out the top most column
    feat = important.iloc[0]
    important_features.append(feat)

In [104]:
important_features = pd.DataFrame(important_features)

In [105]:
important_features.reset_index(inplace=True,drop=True)

In [107]:
important_features.head()

,Features,importance
0,135.0,0.510000
1,128.0,0.563757
2,62.0,0.510000
3,47.0,0.510000
4,183.0,0.285817


In [108]:
len(important_features)

56

In [114]:
feature_consider = set(important_features['Features'])

In [115]:
feature_discard = list(set(corr_list)-set(feature_consider))

In [117]:
len(feature_discard)

87

## Discarding the features that are not important

In [119]:
X_train_grouped_uncorr = X_train_unique.drop(feature_discard,axis=1)
X_test_grouped_uncorr = X_test_unique.drop(feature_discard,axis=1)

In [122]:
X_train_grouped_uncorr.shape,X_test_grouped_uncorr.shape

((16000, 140), (4000, 140))

In [123]:
%%time
run_randomForest(X_train_grouped_uncorr,X_test_grouped_uncorr,y_train,y_test)

Accuracy is :  0.95775
Wall time: 1.94 s
